In [1]:
%load_ext autoreload
%autoreload 2


In [43]:
from domino.slices import collect_settings
from domino.slices.celeba import CelebASliceBuilder
from domino.evaluate import run_sdm, score_sdms, run_sdms
from domino.train import train_settings
import numpy as np
import meerkat as mk
import terra
from domino.emb.clip import embed_phrases

In [27]:
out = run_sdm.inp(run_id=17186)
out

{'data_dp': Artifact(id=221416, run_id=17146, type=<class 'meerkat.datapanel.DataPanel'>),
 'emb_dp': Artifact(id=221123, run_id=16963, type=<class 'meerkat.datapanel.DataPanel'>),
 'model': None,
 'id_column': 'dicom_id',
 'word_dp': Artifact(id=221456, run_id=17179, type=<class 'meerkat.datapanel.DataPanel'>),
 'sdm_class': domino.sdm.gmm.MixtureModelSDM,
 'sdm_config': {'weight_y_log_likelihood': 1, 'n_slices': 5, 'emb': 'emb'},
 'setting_id': 'e70fbf64-4526-4b04-978d-ac9d45e99751',
 'build_setting_run_id': Artifact(id=221465, run_id=17186, type=<class 'numpy.int64'>),
 'score_model_run_id': Artifact(id=221466, run_id=17186, type=<class 'numpy.int64'>),
 'score_settings_run_id': Artifact(id=221467, run_id=17186, type=<class 'numpy.int64'>),
 'synthetic_preds': Artifact(id=221468, run_id=17186, type=<class 'numpy.bool_'>),
 'index': '45',
 'alpha': 0.0,
 'build_setting_kwargs': {'n': 30000,
  'correlate': 'support_devices',
  'target': 'cardiomegaly',
  'corr': 0.0},
 'dataset': 'mim

In [8]:
from meerkat.contrib.mimic import gcs
data_dp = out["data_dp"].load()

In [9]:
out

In [13]:
data_dp["emb"].data

memmap([[ 2.986e-01, -1.765e-01, -2.175e-01, ...,  5.316e-02, -3.538e-01,
          3.516e-01],
        [ 2.986e-01, -1.765e-01, -2.175e-01, ...,  5.316e-02, -3.538e-01,
          3.516e-01],
        [ 2.986e-01, -1.765e-01, -2.175e-01, ...,  5.316e-02, -3.538e-01,
          3.516e-01],
        ...,
        [ 2.510e-01, -2.487e-01, -3.762e-01, ...,  2.196e-01, -3.679e-01,
          4.238e-01],
        [ 2.654e-01, -2.234e-02, -2.629e-01, ...,  2.308e-01, -4.268e-01,
          2.053e-01],
        [ 1.838e-01,  1.221e-04, -3.611e-01, ...,  2.942e-01, -5.239e-01,
          4.797e-01]], dtype=float16)

In [33]:
from domino.emb.clip import embed_images
dp = embed_images.inp(16963)["dp"]
len(dp.load())a

377110

In [35]:
from domino.emb.clip import embed_images
dp = embed_images.out(16963)
len(dp.load())

1914921

In [26]:
from domino.data.mimic import get_mimic_dp, split_dp_preloaded

split_dp_preloaded.out().load()

,split (PandasSeriesColumn),subject_id (PandasSeriesColumn),index (NumpyArrayColumn)
0,train,10000935.0,0.0
1,train,10001176.0,1.0
2,train,10001176.0,2.0
3,train,10001401.0,3.0
4,train,10001401.0,4.0
...,...,...,...
377105,train,19983508.0,377105.0
377106,train,19984491.0,377106.0
377107,train,19991743.0,377107.0
377108,train,19992875.0,377108.0


In [58]:
from domino.utils import merge_in_split
inp = embed_images.inp(16963)
out = merge_in_split(dp=inp["dp"].load(), split_dp=inp["split_dp"].load())

In [59]:
len(out)

377110

In [41]:
inp["split_dp"].load()

,split (PandasSeriesColumn),subject_id (PandasSeriesColumn),index (NumpyArrayColumn)
0,train,10000935.0,0.0
1,train,10001176.0,1.0
2,train,10001176.0,2.0
3,train,10001401.0,3.0
4,train,10001401.0,4.0
...,...,...,...
377105,train,19983508.0,377105.0
377106,train,19984491.0,377106.0
377107,train,19991743.0,377107.0
377108,train,19992875.0,377108.0


In [45]:
out = mk.merge(inp["dp"].load(), inp["split_dp"].load(), on="subject_id")

In [49]:
split_dp = inp["split_dp"].load()
df = split_dp[["subject_id", "split"]].to_pandas()

In [57]:
gb = df.groupby("subject_id")
assert (gb["split"].nunique() == 1).all()
mk.DataPanel.from_pandas(gb["split"].first().reset_index())

,subject_id (PandasSeriesColumn),split (PandasSeriesColumn),index (PandasSeriesColumn)
0,10000032.0,valid,0
1,10000764.0,train,1
2,10000898.0,train,2
3,10000935.0,train,3
4,10000980.0,test,4
...,...,...,...
65374,19999287.0,train,65374
65375,19999376.0,train,65375
65376,19999442.0,test,65376
65377,19999733.0,train,65377
